In [2]:
import requests
import os

In [109]:
# Function that allows us to obtain the dollar price, returning a dictionary with the rate and its date

def obtain_currency(api_key):
    url = "https://api.apilayer.com/exchangerates_data/latest"
    params = { "symbols": "ARS",  
        "base": "USD" }
    headers = {
        "apikey": api_key}
    response = requests.get(url, headers=headers, params=params)
    # Check the result of response, if some failure exist.
    if response.status_code == 200:
        data = response.json()
        cotizacion_dolar = data['rates']['ARS']
        print(f"Cotización actual del dólar: {cotizacion_dolar} ARS")
        registro = { 'date': data['date'] , 'price': data['rates']['ARS']}
        return registro
    else:
        print(f"Error al obtener la cotización: {response.status_code}, {response.text}")



In [6]:
obtain_currency(api_key)


Cotización actual del dólar: 974.732197 ARS


{'date': '2024-10-11', 'price': 974.732197}

In [151]:
import os
import pandas as pd

def append_to_data_price(api_key : str, file : str):
    
    # Get the current record of the dollar price. To then host it as a dictionary, for example: {'date': '2024-10-10', 'price': 750}

    registro = obtain_currency(api_key) 
    
    base_dir = './base_datos/inventario/'
    base_path = Path(base_dir)
    file_path = base_path / file

    # Check if the price file already exists, if not, create an empty DataFrame with the appropriate columns,
    #and then enter the values extracted by the API.
    
    if os.path.exists(file_path):
        data_prices = pd.read_csv(file_path)
    else:
        data_prices = pd.DataFrame(columns=['date', 'price'])

    # Check if the day's record is already in the DataFrame, to avoid duplicating records.
    
    if registro['date'] in data_prices['date'].values:
        print(f"El registro del día {registro['date']} ya existe. No se añadirá de nuevo.")
        return data_prices

    # Convert the record to a DataFrame and add it to data_prices(file).
    
    nuevo_registro = pd.DataFrame([registro])
    data_prices = pd.concat([data_prices, nuevo_registro], ignore_index=True)

    #Save the updated DataFrame to the CSV file.
    
    data_prices.to_csv(file_path, index=False)
    print(f"Record of day {registro['date']} added successfully.")
    
    return data_prices



In [152]:
append_to_data_price(api_key,'data_price.csv')

Cotización actual del dólar: 973.490388 ARS
El registro del día 2024-10-11 ya existe. No se añadirá de nuevo.


,date,price
0,2024-09-09,957.763020
1,2024-09-10,957.430504
2,2024-09-11,957.741278
3,2024-09-12,959.995667
4,2024-09-13,958.043112
5,2024-09-14,959.250402
6,2024-09-15,959.249945
7,2024-09-16,960.448297
8,2024-09-17,961.241401
9,2024-09-18,962.250058


In [7]:
import os
from pathlib import Path

def transform_data (file1: str, file2: str): 
 
    base_dir = './base_datos/'
    base_path = Path(base_dir)
    file1_path = base_path / file1
    file2_path = base_path / file2
    
    # Check that if both files exist.
    
    if file1_path.exists() and file2_path.exists():
        # Read files.
        df1 = pd.read_excel(file1_path,engine='openpyxl')
        df2 = pd.read_csv(file2_path)


        # Get lasted record of prices dataset.
        df2['date'] = pd.to_datetime(df2['date'])
        latest_record = df2.sort_values(by='date').iloc[-1]

        # Transform last record to df.
        latest_record_df = pd.DataFrame([latest_record])  #
        
        # Concancate both df and calculate total amount in pesos.
        df_transform = pd.concat([df1, latest_record_df], ignore_index=True)
        df_transform['date'] = latest_record_df['date'].iloc[0]
        df_transform['price'] = round(df2['price'].iloc[0],2)
        df_transform['total_amount'] = df_transform['price'] * df_transform['Quantity']
                          
        # Save the changes in 'File 1' -->> stock_monetizado.xlsx
        
        df_transform.to_excel(file1_path, index=False)
        
        print(f"The files have been successfully concatenated and saved to {file1_path}")
    else:
        print("One or both files do not exist. Check names and paths.")
    
    

In [10]:
transform_data('stock_monetizado.xlsx','data_price.csv')

The files have been successfully concatenated and saved to base_datos\stock_monetizado.xlsx


In [11]:
df = pd.read_excel('./base_datos/stock_monetizado.xlsx',engine='openpyxl')
df

,PRODUCTO,Modelo,Descripcion,Proveedor,Unnamed: 4,Quantity,Price USD,date,price,total_amount
0,ID-00000001,BSX17,JUEGO DE TUBOS BAHCO 17P CON CRIQUE,ITURRIA S.A,BAHCO,1.0,1.322124,2024-10-06,975.74,975.74
1,ID-00000002,TM12,"TUBO ESTRIADO LARGO DE 1/2"" 12MM",ITURRIA S.A,BAHCO,2.0,0.720865,2024-10-06,975.74,1951.48
2,ID-00000003,TM13,"TUBO ESTRIADO LARGO DE 1/2"" 13MM",ITURRIA S.A,BAHCO,0.0,0.720865,2024-10-06,975.74,0.00
3,ID-00000004,TM14,"TUBO ESTRIADO LARGO DE 1/2"" 14 MM",ITURRIA S.A,BAHCO,0.0,0.720865,2024-10-06,975.74,0.00
4,ID-00000005,TM15,"TUBO ESTRIADO LARGO DE 1/2"" 15 MM",ITURRIA S.A,BAHCO,1.0,0.359955,2024-10-06,975.74,975.74
...,...,...,...,...,...,...,...,...,...,...
9871,ID-00009872,9009018980183,28011,"DISCO CARBURO SILICIO 7""",DELLA VALLE Y CIA SRL,NaN,0.000000,2024-10-06,975.74,NaN
9872,ID-00009873,9009018980480,28012,"DISCO CARBURO SILICIO 3"" P/TALADRO",DELLA VALLE Y CIA SRL,NaN,2.338116,2024-10-06,975.74,NaN
9873,ID-00009874,90511608,89DF-065,TAPA DE CABEZAL B+D GL 800 - GH 750,RC DISTRIBUCIONES S.R.L,NaN,1.857037,2024-10-06,975.74,NaN
9874,ID-00009875,NaN,WGVC - ACUARIO Servicios Informáticos,NaN,NaN,NaN,NaN,2024-10-06,975.74,NaN


In [ ]:
"C:\Users\Novakorp\Desktop\ITBA\data_aplications\etl_airflow\base_datos\stock_monetizado.xlsx"

In [142]:
# Example
mover_archivo_compras_a_inventario('compras_stock.xlsx')

Archivo 'compras_stock.xlsx' movido a inventario correctamente.


In [36]:
fecha_hoy = datetime.today().strftime('%d-%m-%Y')
fecha_hoy


'11-10-2024'

In [143]:
import os
import shutil
from datetime import datetime
from pathlib import Path

def extract_file_purchases_to_stock(archivo):
    # Define directory path.
    base_dir = './base_datos/'
    base_path = Path(base_dir)

    # Define origin and destination paths.
    origin_path = base_path.joinpath('compras')
    destination_path = base_path.joinpath('inventario')

    # Verified if exist origin and destination paths.
    if not origin_path.exists():
        return print(f"El directorio {origin_path} no existe.")
    if not destination_path.exists():
        
        return print(f"El directorio {destination_path} no existe.")

    # Origin path define complete.
    archivo_origen = origin_path.joinpath(archivo)
    
    # Check if the file exists in the defined path.
    if archivo_origen.exists():
        archivo_destino = destination_path.joinpath(archivo)

        # Copy origin file to destination.
        
        shutil.copy(str(archivo_origen), str(archivo_destino))
        print(f"File '{archivo}' moved to stock folder correctly.")
    else:
        print(f"The file '{archivo}' was not found in the purchases directory.")



In [12]:
extract_file_purchases_to_stock('compras_stock.xlsx')

Archivo 'compras_stock.xlsx' movido a inventario correctamente.


In [144]:
import os
import shutil
from datetime import datetime
from pathlib import Path

def extract_file_sells_to_stock(archivo):
    # Define directory path.
    
    base_dir = './base_datos/'
    base_path = Path(base_dir)

    # Define origin and destination paths.
    origin_path = base_path.joinpath('ventas')
    destination_path = base_path.joinpath('inventario')

    # Verified if exist origin and destination paths.
    if not origin_path.exists():
        print(f"Directory {origin_path} not exist.")
        return
    if not destination_path.exists():
        print(f" Directory {destination_path} not exist.")
        return

   # Origin path define complete.
    archivo_origen = origin_path.joinpath(archivo)
    
     # Check if the file exists in the defined path.
    if archivo_origen.exists():
        archivo_destino = destination_path.joinpath(archivo)

        # Copy origin file to destination.
        shutil.copy(str(archivo_origen), str(archivo_destino))
        print(f"File '{archivo}' moved to stock folder correctly.")
    else:
        print(f"File '{archivo}' was not found in the sells directory.")

In [16]:
extract_file_sells_to_stock('ventas_unidades.xlsx')

Archivo 'ventas_unidades.xlsx' movido a inventario correctamente.


In [124]:
import pandas as pd
from pathlib import Path

def update_stock(stock_file: str, compras_file: str, ventas_file: str, fecha: str):
    # Define directory of files.
    base_dir = './base_datos/inventario'
    base_path = Path(base_dir)
    
    # Load files.
    stock_path = base_path / stock_file
    compras_path = base_path / compras_file
    ventas_path = base_path / ventas_file
    
    if stock_path.exists() and compras_path.exists() and ventas_path.exists():
        # Rad stock, purchases and sells files.
        df_stock = pd.read_excel(stock_path, engine='openpyxl')
        df_compras = pd.read_excel(compras_path, engine='openpyxl')
        df_ventas = pd.read_excel(ventas_path, engine='openpyxl')
        
        
       # Convert date columns to datetime format, then match the formats.
        df_stock.columns = pd.to_datetime(df_stock.columns, errors='ignore', dayfirst=True)
        df_compras.columns = pd.to_datetime(df_compras.columns, errors='ignore', dayfirst=True)
        df_ventas.columns = pd.to_datetime(df_ventas.columns, errors='ignore', dayfirst=True)
        
        # Convert date ti datetime format.
        fecha = pd.to_datetime(fecha, format='%Y-%m-%d')

        
        # Check if a date is in a columns of DF.
        if fecha not in df_stock.columns or fecha not in df_compras.columns or fecha not in df_ventas.columns:
            print(f"La fecha {fecha} no está presente en los archivos.{df_stock.columns}")
            return
        
        # Get the column from the day before the last one, this case would be viable if the update of the files we use is done automatically. 
        # As in real life.
        #previous_date = df_stock.columns[-1]
        
        # Update stock = stock of day + units purchases - units sells.
        df_stock[fecha] = df_stock[fecha] + df_compras[fecha] - df_ventas[fecha]
        
        # Save a updated file.
        df_stock.to_excel(stock_path, index=False)
        
        print(f"Stock file has been updated for date {date}.")
    else:
        print("One or more files do not exist. Please check the names and paths.")




In [154]:
# Ejemplo de uso:
update_stock('stock_ferrimac.xlsx', 'compras_stock.xlsx', 'ventas_unidades.xlsx', '2024-09-22')

C:\Users\Novakorp\AppData\Local\Temp\ipykernel_23520\2324201084.py:21: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_stock.columns = pd.to_datetime(df_stock.columns, errors='ignore', dayfirst=True)
C:\Users\Novakorp\AppData\Local\Temp\ipykernel_23520\2324201084.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_stock.columns = pd.to_datetime(df_stock.columns, errors='ignore', dayfirst=True)
C:\Users\Novakorp\AppData\Local\Temp\ipykernel_23520\2324201084.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_compras.columns = pd.to_datetime(df_compras.columns, errors='ignore', dayfirst=True)
C:\Users\Novakorp

El archivo de stock ha sido actualizado para la fecha 2024-09-22 00:00:00.


In [72]:
import requests

url = "https://api.apilayer.com/exchangerates_data/timeseries?start_date=2024-09-09&end_date=2024-10-09"

params = { "symbols": "ARS",  
        "base": "USD" }
headers= {
  "apikey": "dW4BQwNQMHOFjKv73Y6NTi7wocDjt46c"
}

response = requests.request("GET", url, headers=headers, params = params)

status_code = response.status_code
result = response.text

In [74]:
data = response.json()

#cotizacion_dolar = data['rates']['ARS']

In [83]:
data = pd.DataFrame(data)
data

,success,timeseries,start_date,end_date,base,rates
2024-09-09,True,True,2024-09-09,2024-10-09,USD,{'ARS': 957.76302}
2024-09-10,True,True,2024-09-09,2024-10-09,USD,{'ARS': 957.430504}
2024-09-11,True,True,2024-09-09,2024-10-09,USD,{'ARS': 957.741278}
2024-09-12,True,True,2024-09-09,2024-10-09,USD,{'ARS': 959.995667}
2024-09-13,True,True,2024-09-09,2024-10-09,USD,{'ARS': 958.043112}
2024-09-14,True,True,2024-09-09,2024-10-09,USD,{'ARS': 959.250402}
2024-09-15,True,True,2024-09-09,2024-10-09,USD,{'ARS': 959.249945}
2024-09-16,True,True,2024-09-09,2024-10-09,USD,{'ARS': 960.448297}
2024-09-17,True,True,2024-09-09,2024-10-09,USD,{'ARS': 961.241401}
2024-09-18,True,True,2024-09-09,2024-10-09,USD,{'ARS': 962.250058}


In [155]:
dates = []
prices = []
    
    # Go through the data in 'rates' to obtain the rates by date
for date, rates in data['rates'].items():
    dates.append(date)
    prices.append(rates['ARS'])  
# Create df with columns
df = pd.DataFrame({
    'date': dates,
    'price': prices
})

# Print Data Frame
print(df)

          date       price
0   2024-09-09  957.763020
1   2024-09-10  957.430504
2   2024-09-11  957.741278
3   2024-09-12  959.995667
4   2024-09-13  958.043112
5   2024-09-14  959.250402
6   2024-09-15  959.249945
7   2024-09-16  960.448297
8   2024-09-17  961.241401
9   2024-09-18  962.250058
10  2024-09-19  962.754297
11  2024-09-20  961.531104
12  2024-09-21  961.359012
13  2024-09-22  962.642198
14  2024-09-23  965.232700
15  2024-09-24  965.262902
16  2024-09-25  968.500099
17  2024-09-26  967.232698
18  2024-09-27  965.466231
19  2024-09-28  965.466231
20  2024-09-29  965.617504
21  2024-09-30  968.767443
22  2024-10-01  969.592296
23  2024-10-02  970.245698
24  2024-10-03  971.736897
25  2024-10-04  975.742773
26  2024-10-05  975.742773
27  2024-10-06  976.734404
28  2024-10-07  974.829898
29  2024-10-08  974.254402
30  2024-10-09  974.741097


In [104]:
data_prices = df
data_prices

,date,price
0,2024-09-09,957.763020
1,2024-09-10,957.430504
2,2024-09-11,957.741278
3,2024-09-12,959.995667
4,2024-09-13,958.043112
5,2024-09-14,959.250402
6,2024-09-15,959.249945
7,2024-09-16,960.448297
8,2024-09-17,961.241401
9,2024-09-18,962.250058
